In [2]:
from neo4j import GraphDatabase
from sklearn.manifold import TSNE
import numpy as np
import altair as alt
import pandas as pd



In [3]:
driver = GraphDatabase.driver("bolt://localhost/PublicOmics", auth=("neo4j", "ngs4"))


In [13]:
 with driver.session(database="neo4j") as session:
    result = session.run("""
    MATCH (p:Pathway)
    RETURN p.Origin AS origin, p.embeddingNode2vec AS embedding, p.Name AS name
    """)
    X = pd.DataFrame([dict(record) for record in result])

In [14]:
X

,origin,embedding,name
0,Biocarta,"[-0.2407177835702896, -0.07320312410593033, -0...",antisense pathway
1,Biocarta,"[0.6059083342552185, 0.8333665132522583, 0.592...",hemoglobins chaperone
2,Biocarta,"[-0.16733059287071228, -0.48236650228500366, 0...",srebp control of lipid synthesis
3,Biocarta,"[-0.6134916543960571, -0.012800789438188076, 0...",multi-drug resistance factors
4,Biocarta,"[0.9129302501678467, 0.2537408471107483, 0.645...",inhibition of matrix metalloproteinases
...,...,...,...
735,Kegg,"[-0.7549960017204285, 0.816882848739624, 0.963...",basal transcription factors
736,Kegg,"[-0.12670515477657318, -0.413055419921875, 0.3...",snare interactions in vesicular transport
737,Kegg,"[-0.5813363194465637, 0.1551935374736786, -0.0...",retinol metabolism
738,Kegg,"[0.04845542460680008, -0.6593214869499207, -0....",lipoic acid metabolism


In [15]:
X_embedded = TSNE(n_components=2, random_state=6).fit_transform(list(X.embedding))

df = pd.DataFrame(data = {
    "pathway": X.name,
    "Source": X.origin,
    "x": [value[0] for value in X_embedded],
    "y": [value[1] for value in X_embedded]
})

In [19]:
# df
alt.Chart(df).mark_circle(size=50).encode(
    x='x',
    y='y',
    color='Source',
    tooltip=['pathway', 'Source', 'x','y']
).properties(width=600, height=400)

alt.Chart(...)